<a href="https://colab.research.google.com/github/sujh8/AirBnB_clone/blob/main/translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets  # Install the 'datasets' module
from datasets import load_dataset
import pandas as pd
# Load the dataset
ds = load_dataset("odunola/french-english-unprocessed")
# Convert to DataFrame and inspect
df = pd.DataFrame(ds['train'])
print(df.head())

                                            sentence  \
0                          bon on va tout de suite\n   
1  croyez-vous qu' il y a un docteur ou une infir...   
2                      c' est à droite ou à gauche\n   
3                   merci bien de votre assistance\n   
4            est-ce que le médecin est loin d' ici\n   

                                               audio  \
0  {'path': 'afc-gabon_16.06.12_023_read_0085.wav...   
1  {'path': 'afc-gabon_16.06.12_023_read_0084.wav...   
2  {'path': 'afc-gabon_16.06.12_023_read_0086.wav...   
3  {'path': 'afc-gabon_16.06.12_023_read_0087.wav...   
4  {'path': 'afc-gabon_16.06.12_023_read_0083.wav...   

                          english_transcript  
0             All right, let's go right now.  
1  Do you think there's a doctor or a nurse?  
2          It's on the right or on the left.  
3    Thank you very much for your assistance  
4               Is the doctor far from here?  


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from transformers import MarianTokenizer
from datasets import Dataset,DatasetDict
!pip install sacremoses
!pip install scikit-learn
# Load tokenizer
from sklearn.model_selection import train_test_split
tokenizer = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-fr-en')
# Preprocessing function
def preprocess_function(examples):
    inputs = examples['sentence']
    targets = examples['english_transcript']
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding='max_length')
    labels = tokenizer(targets, max_length=128, truncation=True, padding='max_length')
    model_inputs['labels'] = labels['input_ids']
    return model_inputs
# Use a smaller subset for tokenization
df_subset = df.head(1000)
train_data, val_data = train_test_split(df_subset, test_size=0.1)
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)
dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset
})
# Tokenize the datasets
tokenized_train = dataset_dict['train'].map(preprocess_function, batched=True)
tokenized_eval = dataset_dict['validation'].map(preprocess_function, batched=True)

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
import tensorflow as tf
from transformers import TFMarianMTModel, DataCollatorForSeq2Seq
try:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(resolver)
    tf.config.experimental_initialize_former_session(resolver)
    strategy = tf.distribute.TPUStrategy(resolver)
except ValueError:
    print("Not connected to a TPU runtime. Running with CPU/GPU.")
    strategy = tf.distribute.get_strategy()
# Load model and compile
with strategy.scope():
    model = TFMarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-fr-en')
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
    model.compile(optimizer=optimizer)
    # Initialize data collator
    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors='tf')
    # Convert datasets to TensorFlow tensors
    tf_train_dataset = tokenized_train.to_tf_dataset(
        columns=['input_ids', 'attention_mask', 'labels'],
        shuffle=True,
        batch_size=8,  # Adjust batch size as needed
        collate_fn=data_collator
    )
    tf_eval_dataset = tokenized_eval.to_tf_dataset(
        columns=['input_ids', 'attention_mask', 'labels'],
        shuffle=False,
        batch_size=8,  # Adjust batch size as needed
        collate_fn=data_collator
    )
    # Train the model
    model.fit(tf_train_dataset, validation_data=tf_eval_dataset, epochs=1)
    metrics = model.evaluate(tf_eval_dataset)
    print("Evaluation metrics:", metrics)
    # Save the model
    model.save('./saved_models/tf_model', save_format='tf')

Not connected to a TPU runtime. Running with CPU/GPU.


All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-fr-en.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
13/13 [==============================] - 91s 7s/step - loss: 2.7125
Evaluation metrics: 2.7125463485717773


In [ ]:
import zipfile
import os

model_path = './saved_models/tf_model'

# Create a zip file
with zipfile.ZipFile('my_model.zip', 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, _, files in os.walk(model_path):
        for file in files:
            zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), model_path))